## 2. Получение предсказаний на тестовой выборке и создание файла для kaggle.

Считаем данные из яндекс-диска

In [2]:
import os
import pandas as pd
path = r'kaggle_data' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    #for dir_entry in os.listdir(path):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

kaggle_data/track_0.csv
kaggle_data/track_1.csv
kaggle_data/track_10.csv
kaggle_data/track_100.csv
kaggle_data/track_101.csv
kaggle_data/track_102.csv
kaggle_data/track_103.csv
kaggle_data/track_104.csv
kaggle_data/track_105.csv
kaggle_data/track_106.csv
kaggle_data/track_107.csv
kaggle_data/track_108.csv
kaggle_data/track_109.csv
kaggle_data/track_11.csv
kaggle_data/track_110.csv
kaggle_data/track_111.csv
kaggle_data/track_112.csv
kaggle_data/track_113.csv
kaggle_data/track_114.csv
kaggle_data/track_115.csv
kaggle_data/track_116.csv
kaggle_data/track_117.csv
kaggle_data/track_118.csv
kaggle_data/track_119.csv
kaggle_data/track_12.csv
kaggle_data/track_120.csv
kaggle_data/track_121.csv
kaggle_data/track_122.csv
kaggle_data/track_123.csv
kaggle_data/track_124.csv
kaggle_data/track_125.csv
kaggle_data/track_126.csv
kaggle_data/track_127.csv
kaggle_data/track_128.csv
kaggle_data/track_129.csv
kaggle_data/track_13.csv
kaggle_data/track_130.csv
kaggle_data/track_131.csv
kaggle_data/track_13

Переведём типы движений в числа

In [3]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [4]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [5]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):
    
# обрезаем начало и конец трека
    if a == -1:
        st = 1250
    else:
        st = a
            
    if b == -1:
        end = -1250
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1

    print(name)
#     fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))

# приводим данные с Iphone к нужному формату
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return -1

# рисуем графики ускорений
#     if cut:
#         ax1.plot(df.iloc[st:end]['gFx'],c='g')
#         ax1.plot(df.iloc[st:end]['gFy'],c='r')
#         ax1.plot(df.iloc[st:end]['gFz'],c='b')
#     else:
#         ax1.plot(df['gFx'],c='g')
#         ax1.plot(df['gFy'],c='r')
#         ax1.plot(df['gFz'],c='b')
        
# вычисляем модуль (норму) ускорения и рисуем его
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
#     ax2.plot(Acc)
#     plt.show()

# можно различить многие действия по амплитуде ускорения
    amplitude = Acc.max() - Acc.min()
    print(' АМПЛИТУДА:', Acc.max()-Acc.min())
    print()
    
    return amplitude

Ampl = np.zeros(len(Data)) 

for df, target in zip(Data, Target):
    track_num = int(target.split('_')[2].split('.')[0]) # важный момент, что треки не по нумерации идут!
    Ampl[track_num] = process_data(df, name=target)

Populating the interactive namespace from numpy and matplotlib
kaggle_data/track_0.csv


/home/yan/anaconda3/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


 АМПЛИТУДА: 1.2409331302405158

kaggle_data/track_1.csv
 АМПЛИТУДА: 1.9935598883036225

kaggle_data/track_10.csv
 АМПЛИТУДА: 7.802304392990605

kaggle_data/track_100.csv
 АМПЛИТУДА: 0.34113272676839523

kaggle_data/track_101.csv
 АМПЛИТУДА: 0.4539765748076483

kaggle_data/track_102.csv
 АМПЛИТУДА: 2.2110203625988167

kaggle_data/track_103.csv
 АМПЛИТУДА: 3.1345378975461413

kaggle_data/track_104.csv
 АМПЛИТУДА: 1.5750179236901383

kaggle_data/track_105.csv
 АМПЛИТУДА: 2.5835429119223496

kaggle_data/track_106.csv
 АМПЛИТУДА: 0.6312015938792386

kaggle_data/track_107.csv
 АМПЛИТУДА: 2.124336238883378

kaggle_data/track_108.csv
 АМПЛИТУДА: 3.1243961620554765

kaggle_data/track_109.csv
 АМПЛИТУДА: nan

kaggle_data/track_11.csv
 АМПЛИТУДА: 1.4499291676896982

kaggle_data/track_110.csv
 АМПЛИТУДА: 1.8330376632404348

kaggle_data/track_111.csv
 АМПЛИТУДА: 0.042436878839561376

kaggle_data/track_112.csv
 АМПЛИТУДА: 1.4825475120699383

kaggle_data/track_113.csv
 АМПЛИТУДА: 0.7902169184700258



In [44]:
%pylab inline

def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / numpy.linalg.norm(vec1)).reshape(3), (vec2 / numpy.linalg.norm(vec2)).reshape(3)
    v = numpy.cross(a, b)
    if any(v): #if not all zeros then 
        c = numpy.dot(a, b)
        s = numpy.linalg.norm(v)
        kmat = numpy.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return numpy.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3) #cross of all zeros only occurs on identical directions

def rotate(X, Y, Z, Mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(X,Y,Z):
        res = Mat.dot(elem)
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

def integrate(lst):
    res = []
    s = 0
    for elem in lst:
        s += elem
        res.append(s)
    return res

def process_data_fft(df, comma=True, cut=True, name = "", a=-1, b=-1):
    
    l = df.shape[0]
    if a == -1:
        st = l // 5
    else:
        st = a
            
    if b == -1:
        end = - l // 5
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1

    print(name)
    #fig, ((ax1, ax2, ax3),(ax4, ax5, ax6),(ax7, ax8, ax9)) = plt.subplots(nrows=3, ncols=3, figsize=(15,10))
    
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        df['gFx'] = df['gFx'].astype(float)
        df['gFy'] = df['gFy'].astype(float)
        df['gFz'] = df['gFz'].astype(float)
#     if cut:
#         ax1.plot(df.iloc[st:end]['gFx'],c='g')
#         ax1.plot(df.iloc[st:end]['gFy'],c='r')
#         ax1.plot(df.iloc[st:end]['gFz'],c='b')
#     else:
#         ax1.plot(df['gFx'],c='g')
#         ax1.plot(df['gFy'],c='r')
#         ax1.plot(df['gFz'],c='b')
#        show()
########################################################################        
#integration

#вычисляем средний вектор ускорения по треку

    mx = df.iloc[st:end]['gFx'].values.mean() 
    my = df.iloc[st:end]['gFy'].values.mean() 
    mz = df.iloc[st:end]['gFz'].values.mean() 
    
#находим матрицу поворота этого вектора к вектору (0,0,1)

    Vec = [mx,my,mz]
    Point = [0, 0, 1]
    mat = rotation_matrix_from_vectors(Vec, Point)
    
#каждую точку трека поворачиваем на эту матрицу

    new_x, new_y, new_z = rotate(df.iloc[st:end]['gFx'].values, df.iloc[st:end]['gFy'].values, df.iloc[st:end]['gFz'].values, mat)

#проверяем, что новые средние равны примерно (0,0,1). Если это не так, значит, телефон часто поворачивался при движении, и нам не поможет это преобразование

    #print('New means:', mean(new_x), mean(new_y), mean(new_z))

#делим на 250*250 (это нужно было еще при интегрировании)
    new_x = [elem / 62500 for elem in new_x]
    new_y = [elem / 62500 for elem in new_y]
#из ускорения по z вычитаем 1 (g), чтобы убрать влияние силы тяжести и оставить только ускорение по z
    new_z = [(elem-1)/62500 for elem in new_z]
    
#дважды интегрируем ускорение, чтобы получить перемещение вдоль каждой оси
    int_x = integrate(integrate(new_x))
  #  ax7.plot(int_x)
    
    int_y = integrate(integrate(new_y))
  #  ax8.plot(int_y)
    
    int_z = integrate(integrate(new_z))
#     ax9.plot(int_z)

########################################################################        
#преобразование Фурье
########################################################################        

    Fs = 250 #частота сбора данных
    y = df.iloc[st:end]['gFx'].values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    
    Min = 0
    
#     ax4.plot(frq,yabs) # plotting the spectrum
#     ax4.set_xlim([0,30]) #очень большие частоты нам не нужны
#     ax4.set_ylim([Min,0.1]) #не смотрим на близкие к нулю величины частот

    
    y = df.iloc[st:end]['gFy'].values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    


    y = df.iloc[st:end]['gFz'].values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    #print(frq)
   
    #ax6.plot(frq, yabs) # plotting the spectrum
    #ax6.set_xlim([0,30])
    #ax6.set_ylim([Min,0.1])


    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

    y = Acc.values
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    Y = np.fft.fft(y)/n # dft and normalization
    Y = Y[:n//2]
    
    yabs = abs(Y)
    print('==================')
    n_yabs = list(filter(lambda x: x <= 0.1, yabs))[8:150]
    print("MEDIAN:", np.median(n_yabs))
    print("MEAN:", np.mean(n_yabs))
    print("MAX:", max(n_yabs))
    #ax3.plot(frq,yabs) # plotting the spectrum
    #ax3.set_xlim([0,30])
    #ax3.set_ylim([Min,0.1])
########################################################################        
    
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
    #ax2.plot(Acc)
    #plt.show()
    
    print(name + ' Амплитуда:', Acc.max()-Acc.min())
    
    return [Acc.max()-Acc.min(), np.mean(n_yabs), max(n_yabs)]

count = 0
Metrics = [[] for i in range(len(Data))]
# 0 - ampl
# 1 - mean
# 2 - max val
for df, target in zip(Data, Target):
    track_num = int(target.split('_')[2].split('.')[0]) # важный момент, что треки не по нумерации идут!
    Metrics[track_num] = process_data_fft(df, name=target)
    
'''

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}
'''

Populating the interactive namespace from numpy and matplotlib
kaggle_data/track_0.csv
MEDIAN: 0.00617249243397948
MEAN: 0.00936301961504372
MAX: 0.09508578626555722
kaggle_data/track_0.csv Амплитуда: 2.617663370775875
kaggle_data/track_1.csv
MEDIAN: 0.005477546818507818
MEAN: 0.010280740742380319
MAX: 0.0843664734132951
kaggle_data/track_1.csv Амплитуда: 1.9935598883036225
kaggle_data/track_10.csv
MEDIAN: 0.021312672969362245
MEAN: 0.026098485683654117
MAX: 0.09223228858795274
kaggle_data/track_10.csv Амплитуда: 8.833789142567072
kaggle_data/track_100.csv
MEDIAN: 3.245093673276883e-05
MEAN: 3.598071101203253e-05
MAX: 9.528167023911715e-05
kaggle_data/track_100.csv Амплитуда: 0.03572333647636561
kaggle_data/track_101.csv
MEDIAN: 0.0016398669134211348
MEAN: 0.001889995815841213
MAX: 0.007303152193069663
kaggle_data/track_101.csv Амплитуда: 0.4539765748076483
kaggle_data/track_102.csv
MEDIAN: 0.011132637047910992
MEAN: 0.011963690897271709
MAX: 0.04069762722858947
kaggle_data/track_102.c

MEDIAN: 0.006648447388126387
MEAN: 0.00685217061512469
MAX: 0.017842073041638273
kaggle_data/track_14.csv Амплитуда: 2.4185341114208763
kaggle_data/track_140.csv
MEDIAN: 0.021053915270891836
MEAN: 0.025513808114924794
MAX: 0.09199546271051766
kaggle_data/track_140.csv Амплитуда: 11.036229463203723
kaggle_data/track_141.csv
MEDIAN: 8.350914888218641e-05
MEAN: 0.00010024037729308762
MAX: 0.000560296940441056
kaggle_data/track_141.csv Амплитуда: 0.01855062925142026
kaggle_data/track_142.csv
MEDIAN: 0.003596404318826223
MEAN: 0.0037580790145092346
MAX: 0.007859458198820161
kaggle_data/track_142.csv Амплитуда: 3.1563878824421536
kaggle_data/track_143.csv
MEDIAN: 0.0011009842101198058
MEAN: 0.0016622014459058971
MAX: 0.009525133936307948
kaggle_data/track_143.csv Амплитуда: 0.4194272222654043
kaggle_data/track_144.csv
MEDIAN: 0.008725246886448166
MEAN: 0.010622063898009425
MAX: 0.02919199229599914
kaggle_data/track_144.csv Амплитуда: 2.493435844215357
kaggle_data/track_145.csv
MEDIAN: 0.0299

MEDIAN: 0.0006220983855618029
MEAN: 0.00075027476833089
MAX: 0.003293073886125048
kaggle_data/track_53.csv Амплитуда: 0.10877474716784707
kaggle_data/track_54.csv
MEDIAN: 0.002606638340880791
MEAN: 0.004444504096318121
MAX: 0.05442972972121134
kaggle_data/track_54.csv Амплитуда: 0.8538493326402375
kaggle_data/track_55.csv
MEDIAN: 0.004523195415253624
MEAN: 0.007625464953923209
MAX: 0.0931702822603988
kaggle_data/track_55.csv Амплитуда: 8.830140717365287
kaggle_data/track_56.csv
MEDIAN: 0.009990644167428876
MEAN: 0.010839904119865537
MAX: 0.033029823492647395
kaggle_data/track_56.csv Амплитуда: 9.293501190500901
kaggle_data/track_57.csv
MEDIAN: 0.00643131372328088
MEAN: 0.00841261840030115
MAX: 0.028013416647558957
kaggle_data/track_57.csv Амплитуда: 2.6278376772731593
kaggle_data/track_58.csv
MEDIAN: 0.00745665690341493
MEAN: 0.008734769954927387
MAX: 0.040492065155451784
kaggle_data/track_58.csv Амплитуда: 1.2550040737052317
kaggle_data/track_59.csv
MEDIAN: 0.0011701038126473518
MEAN:

MEDIAN: 0.0014760627847146585
MEAN: 0.0017435948494762177
MAX: 0.006799403704794583
kaggle_data/track_96.csv Амплитуда: 2.8290340370565588
kaggle_data/track_97.csv
MEDIAN: 0.004476594448902088
MEAN: 0.006851416884878686
MAX: 0.02901309899436155
kaggle_data/track_97.csv Амплитуда: 2.3646269061128375
kaggle_data/track_98.csv
MEDIAN: 0.00016013898957574014
MEAN: 0.00022439653299064627
MAX: 0.0006777795501537844
kaggle_data/track_98.csv Амплитуда: 0.3832853345388927
kaggle_data/track_99.csv
MEDIAN: 0.0052288661385987605
MEAN: 0.006888316126917319
MAX: 0.061510711463791404
kaggle_data/track_99.csv Амплитуда: 1.1246967604472298


"\n\nanswers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, \n           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}\n"

Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [70]:
import random
def get_prediction(ampl, mean, maximum):
    if ampl < 1:
        return 0 #стояние
    if ampl >  5:
        return 2 #бег
    else:
        if maximum <= 0.055:
            return 3
        else:
            return random.choice([1, 4, 1, 1])

In [71]:
len(Data), len(Metrics), len(Target)

(146, 146, 146)

In [72]:
type(Data)

list

In [73]:
Metrics_new = []

i=0
for elem in Metrics:
    if not (math.isnan(elem[0]) or elem[0] == -1):
        i += 1
        Metrics_new.append(elem)
    else:
        Metrics_new.append(2)
len(Metrics_new), i

(146, 146)

Создадим таблицу, содержащую номер трека и предсказание.

In [74]:
prediction = [get_prediction(a, b, c) for [a, b, c] in Metrics]

Res = pd.DataFrame({'track_num' : np.arange(len(Data)), 'action' : prediction})
Res.head(10)

,track_num,action
0,0,1
1,1,1
2,2,3
3,3,2
4,4,1
5,5,3
6,6,3
7,7,2
8,8,0
9,9,0


In [75]:
Res.to_csv("first_submit.csv", index=False)